# Switchover Times Basic Example

This example demonstrates queueing systems with switchover times between different service modes.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.STD)

// Create model with switchover times
val model = Network("SwitchoverModel")

// Create nodes
val source = Source(model, "Source")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
val sink = Sink(model, "Sink")

// Set server configurations
queue1.setNumberOfServers(1)
queue2.setNumberOfServers(1)

In [ ]:
// Create job classes
val jobclass1 = OpenClass(model, "Class1")
val jobclass2 = OpenClass(model, "Class2")

// Set arrival rates
source.setArrival(jobclass1, Exp.fitMean(1.0))
source.setArrival(jobclass2, Exp.fitMean(1.2))

In [ ]:
// Set service times
queue1.setService(jobclass1, Exp.fitMean(2.0))
queue1.setService(jobclass2, Exp.fitMean(1.8))

queue2.setService(jobclass1, Exp.fitMean(1.5))
queue2.setService(jobclass2, Exp.fitMean(2.2))

// Note: Switchover times would be modeled through
// specialized scheduling strategies or state-dependent routing
// This is a simplified representation

In [ ]:
// Set routing with probabilistic switching
model.addLink(source, queue1)
model.addLink(source, queue2)
model.addLink(queue1, sink)
model.addLink(queue2, sink)

// Set routing probabilities
val P = model.initRoutingMatrix()
P.set(jobclass1, jobclass1, source, queue1, 0.6)
P.set(jobclass1, jobclass1, source, queue2, 0.4)
P.set(jobclass2, jobclass2, source, queue1, 0.3)
P.set(jobclass2, jobclass2, source, queue2, 0.7)
model.link(P)

In [ ]:
// Solve with different solvers
val solvers = mutableListOf<Pair<String, NetworkSolver>>()

// JMT solver (good for complex models)
try {
    val solverJmt = JMT(model)
    solverJmt.setOptions("samples", 100000)
    solvers.add("JMT" to solverJmt)
} catch (e: Exception) {
    println("JMT solver not available: ${e.message}")
}

// CTMC solver
try {
    val solverCtmc = CTMC(model)
    solverCtmc.setOptions("cutoff", 20)
    solvers.add("CTMC" to solverCtmc)
} catch (e: Exception) {
    println("CTMC solver error: ${e.message}")
}

In [ ]:
// Print results for all solvers
for ((name, solver) in solvers) {
    try {
        println("\n=== $name Solver Results ===")
        val avgTable = solver.avgTable
        avgTable.print()
        
        // Analyze switchover effects
        val queue1Util = avgTable.getUtil().get(0, 1) + avgTable.getUtil().get(1, 1)
        val queue2Util = avgTable.getUtil().get(0, 2) + avgTable.getUtil().get(1, 2)
        val queue1RespT = (avgTable.getRespT().get(0, 1) + avgTable.getRespT().get(1, 1)) / 2.0
        val queue2RespT = (avgTable.getRespT().get(0, 2) + avgTable.getRespT().get(1, 2)) / 2.0
        
        println("\nSwitchover Analysis:")
        println("Queue1 Total Utilization: $queue1Util")
        println("Queue2 Total Utilization: $queue2Util")
        println("Queue1 Average Response Time: $queue1RespT")
        println("Queue2 Average Response Time: $queue2RespT")
        println("Load Balance Ratio: ${queue2Util / queue1Util}")
        
    } catch (e: Exception) {
        println("$name solver failed: ${e.message}")
    }
}